# Customize Chat model
* Building a chat model for conversation
* reference
    * https://langchain.readthedocs.io/en/latest/modules/agents/examples/chat_conversation_agent.html
    https://note.com/npaka/n/n6b7a07e492f1

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/langchain/blob/main/examples/customize_chat_model.ipynb)

In [2]:
%%capture
!pip install langchain
!pip install google-search-results
!pip install openai

In [3]:
from langchain.utilities import SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

## ツール一覧
* python_relp
    * pythonコマンドを実行できる．
* serpapi
    * サーチエンジン，現在のイベントに関する質問に答える必要がある場合に役に立つ
* requests
    * サイトから特定のコンテンツを取得する場合に使う
    * 入力はURLで出力はそのページの全てのテキスト
* terminal
    * ターミナルでのコマンド実行
* llm-math
    * 数学の質問に答えるよう
* pal-math
    * 数学の質問に答えるよう
* pal-colored-objects
    * オブジェクトの位置と色についての推論に優れた言語モデル
* open-meteo-api
    * 気象情報を取得したい場合に使う
* news-api
    * 現在のニュース記事のトップ，ヘッドラインに関する情報を取得する
* tmdb-api
    * The Movie Databaseから情報を取得するのに使う

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

In [ ]:
# https://langchain.readthedocs.io/en/latest/modules/agents/implementations/react.html

from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents.react.base import DocstoreExplorer
docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm = OpenAI(temperature=0, model_name="text-davinci-002")
react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)

In [ ]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="conversational-react-description", 
    verbose=True
)

input = {}

agent.run("興味を引くようなブレインパッドの広告を10個作ってください")

In [10]:
search = SerpAPIWrapper()

tools = [
    Tool(
        name = "Current Search",
        func=search.run,
        description="useful for when you need to answer questions about current events or the current state of the world. the input to this should be a single search term."
    ),
]

# モデルの読み込み
llm=ChatOpenAI(temperature=0)

# 記憶の追加
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

agent = initialize_agent(
    tools, 
    llm, 
    agent="chat-conversational-react-description", 
    verbose=True, 
    memory=memory
    )

## エージェント一覧
* https://langchain.readthedocs.io/en/latest/modules/agents/agents.html
* どのアクションをどのような順序で行うかを決めるもの
* zero-shot-react-description
    * 
* react-docstore
    * ドキュメントベースでの回答をする
* self-ask-with-search
    * 質問に対する事実に基づく回答を検索できる
* conversational-react-description
    * 会話用に最適化されたエージェント
    * 以前の会話のやり取りを記憶できる

In [20]:
agent.run(input="株式会社ブレインパッドについて教えてください")



> Entering new AgentExecutor chain...
```json
{
    "action": "Current Search",
    "action_input": "株式会社ブレインパッド 最新情報"
}
```
Observation: ブレインパッドは、2004年の創業以来 変革を目指す企業と共に最前線を走り続ける、データ活用推進パートナーの ... ブレインパッドの最新情報をお知らせします。
Thought:```json
{
    "action": "Final Answer",
    "action_input": "株式会社ブレインパッドは、データ活用推進パートナーとして、変革を目指す企業と共に最前線を走り続けています。最新情報については、具体的な情報が見つかりませんでした。"
}
```

> Finished chain.


'株式会社ブレインパッドは、データ活用推進パートナーとして、変革を目指す企業と共に最前線を走り続けています。最新情報については、具体的な情報が見つかりませんでした。'